# **LaTeX table with the BAO-fit results**

This notebook shows how to create a LaTeX table with the BAO-fit results varying the settings

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
code_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
save_path = f"{os.environ['PSCRATCH']}/BAOfit_wtheta"
sys.path.append(code_path)
import numpy as np
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"
from matplotlib.ticker import MaxNLocator
from utils_data import GetThetaLimits
from utils_baofit import BAOFitInitializer
from utils_cosmology import CosmologicalParameters

old_template = "_old"
# old_template = ""

def rework_dataset_name(dataset):
    if dataset == "DESY6_dec_below-23.5":
        return r"\texttt{Deccut_noDESI}"
    elif dataset == "DESY6_dec_above-23.5":
        return r"\texttt{Deccut_DESIonly}"
    elif dataset == "DESY6_DR1tiles_noDESI":
        return r"\texttt{DR1tiles_noDESI}"
    elif dataset == "DESY6_DR1tiles_DESIonly":
        return r"\texttt{DR1tiles_DESIonly}"
    elif dataset == "DESY6":
        return "Full sample"
    else:
        return dataset

dataset_list = ["DESY6", "DESY6_DR1tiles_noDESI", "DESY6_DR1tiles_DESIonly", "DESY6_dec_below-23.5", "DESY6_dec_above-23.5"]
include_wiggles_list = ["y", "n"]
# include_wiggles_list = ["y"]
weight_type_list = [1, 0]
# weight_type_list = [1]
bins_removed_list = [[], [0], [1], [2], [3], [4], [5], [1, 2, 3, 4, 5], [0, 2, 3, 4, 5], 
                     [0, 1, 3, 4, 5], [0, 1, 2, 4, 5], [0, 1, 2, 3, 5], [0, 1, 2, 3, 4]]
# bins_removed_list = [[]]

nz_flag = "fid"
dynamical_theta_limits = False

fit_results = {}
likelihoods = {}

for dataset, include_wiggles, weight_type, bins_removed in product(dataset_list, include_wiggles_list, weight_type_list, bins_removed_list):
    theta_min, theta_max = GetThetaLimits(dataset=dataset, nz_flag=nz_flag, dynamical_theta_limits=dynamical_theta_limits, code_path=code_path).get_theta_limits()

    # 1. Arguments
    class Args:
        def __init__(self):
            self.include_wiggles = include_wiggles
            self.dataset = dataset
            self.weight_type = weight_type
            self.mock_id = "mean" # it will not be used...
            self.nz_flag = nz_flag
            self.cov_type = "cosmolike"
            self.cosmology_template = "planck" + old_template
            self.cosmology_covariance = "planck"
            self.delta_theta = 0.2
            self.theta_min = theta_min
            self.theta_max = theta_max
            self.pow_broadband = [-2, -1, 0]
            self.bins_removed = bins_removed
            self.diag_only = "n"
            self.remove_crosscov = "n"
            self.alpha_min = 0.8
            self.alpha_max = 1.2
            self.code_path = code_path
            self.save_path = save_path
    args = Args()
    args.include_wiggles = "" if args.include_wiggles == "y" else "_nowiggles"
    
    # 2. BAO fit initializer. This basically creates the path to load the results
    baofit_initializer = BAOFitInitializer(
        include_wiggles=args.include_wiggles,
        dataset=args.dataset,
        weight_type=args.weight_type,
        mock_id=args.mock_id,
        nz_flag=args.nz_flag,
        cov_type=args.cov_type,
        cosmology_template=args.cosmology_template,
        cosmology_covariance=args.cosmology_covariance,
        delta_theta=args.delta_theta,
        theta_min=args.theta_min,
        theta_max=args.theta_max,
        pow_broadband=args.pow_broadband,
        bins_removed=args.bins_removed,
        alpha_min=args.alpha_min,
        alpha_max=args.alpha_max,
        verbose=False,
        save_path=args.save_path,
    )

    fit_results[dataset, include_wiggles, weight_type, str(bins_removed)] = np.loadtxt(os.path.join(baofit_initializer.path_baofit, "fit_results.txt"))
    likelihoods[dataset, include_wiggles, weight_type, str(bins_removed)] = np.loadtxt(os.path.join(baofit_initializer.path_baofit, "likelihood_data.txt"))


In [ ]:
cosmology_params = CosmologicalParameters(args.cosmology_template, verbose=True)
cosmo = cosmology_params.get_cosmology()

DM_fid = cosmo.comoving_angular_distance(0.85) / cosmo.h
rd_fid = cosmo.rs_drag / cosmo.h
factor = DM_fid / rd_fid

systematic_error = np.sqrt(0.0035**2 + 0.0058**2)
# systematic_error = np.sqrt(0.0045**2 + 0.0057**2)

data = []
for key, values in fit_results.items():
    dataset, include_wiggles, weight_type, bins_removed = key
    alpha, sigma_alpha, chi2, dof = values

    total_sigma_alpha = np.sqrt(sigma_alpha**2 + systematic_error**2)
    alpha_str = f"${alpha:.4f} \\pm {total_sigma_alpha:.4f}$"
    dm_rd_str = f"${alpha * factor:.2f} \\pm {total_sigma_alpha * factor:.2f}$"
    chi2_dof_str = f"{chi2:.1f}/{int(round(dof))}"

    data.append([dataset, include_wiggles, weight_type, bins_removed, alpha_str, dm_rd_str, chi2_dof_str])

df = pd.DataFrame(data, columns=[
    "Dataset", "Wiggles", "Weight Type", "Bins Removed", 
    r"$\alpha \pm \sigma_\alpha$", r"$D_M/r_d$", r"$\chi^2/\mathrm{dof}$"
])

latex_table = df.to_latex(index=False, escape=False, column_format="ccc|ccc")

print(latex_table)


In [ ]:
# Extract the full sample
alpha_full, sigma_alpha_full, _, _ = fit_results["DESY6", "y", 1, "[]"]
print(f"[Full sample] α = {alpha_full:.4f} ± {sigma_alpha_full:.4f}")

# ---- DR1 tiles ----
alpha_1, sigma_alpha_1, _, _ = fit_results["DESY6_DR1tiles_noDESI", "y", 1, "[]"]
alpha_2, sigma_alpha_2, _, _ = fit_results["DESY6_DR1tiles_DESIonly", "y", 1, "[]"]

# Combine
alpha_combined = (alpha_1 / sigma_alpha_1**2 + alpha_2 / sigma_alpha_2**2) / (1 / sigma_alpha_1**2 + 1 / sigma_alpha_2**2)
sigma_alpha_combined = (1 / (1 / sigma_alpha_1**2 + 1 / sigma_alpha_2**2))**0.5

delta = alpha_combined - alpha_full
delta_sigma = (sigma_alpha_combined**2 + sigma_alpha_full**2)**0.5
significance = delta / delta_sigma

print(f"[DR1 tiles] Combined α = {alpha_combined:.4f} ± {sigma_alpha_combined:.4f}")
print(f"Difference from full sample: Δα = {delta:.4f} ± {delta_sigma:.4f} ({significance:.2f}σ)")

# ---- Dec split ----
alpha_1, sigma_alpha_1, _, _ = fit_results["DESY6_dec_below-23.5", "y", 1, "[]"]
alpha_2, sigma_alpha_2, _, _ = fit_results["DESY6_dec_above-23.5", "y", 1, "[]"]

# Combine
alpha_combined = (alpha_1 / sigma_alpha_1**2 + alpha_2 / sigma_alpha_2**2) / (1 / sigma_alpha_1**2 + 1 / sigma_alpha_2**2)
sigma_alpha_combined = (1 / (1 / sigma_alpha_1**2 + 1 / sigma_alpha_2**2))**0.5

delta = alpha_combined - alpha_full
delta_sigma = (sigma_alpha_combined**2 + sigma_alpha_full**2)**0.5
significance = delta / delta_sigma

print(f"[Dec split] Combined α = {alpha_combined:.4f} ± {sigma_alpha_combined:.4f}")
print(f"Difference from full sample: Δα = {delta:.4f} ± {delta_sigma:.4f} ({significance:.2f}σ)")


In [ ]:
import scipy

def chi2_sigma_region(alpha, chi2, threshold=1):
    best = np.argmin(chi2)
    alpha_best = alpha[best]
    chi2_best = chi2[best]

    alpha_down = None
    alpha_up = None

    for i in range(best - 1, -1, -1):
        if chi2[i] < chi2_best + threshold and chi2[i - 1] > chi2_best + threshold:
            alpha_down = alpha[i]
            break

    for i in range(best, len(alpha) - 1):
        if chi2[i] < chi2_best + threshold and chi2[i + 1] > chi2_best + threshold:
            alpha_up = alpha[i]
            break

    if alpha_down is None or alpha_up is None:
        raise ValueError("Could not find both threshold crossings.")

    sigma = (alpha_up - alpha_down) / 2

    return alpha_best, sigma

def chi2_shift_stretch(alpha, chi2, delta_alpha=0, sigma_alpha=1):
    
    # shift
    
    chi2_interp = scipy.interpolate.Akima1DInterpolator(alpha, chi2)
    chi2_shift = chi2_interp(alpha - delta_alpha, extrapolate=False)
    chi2_shift[np.isnan(chi2_shift)] = 10**10
    
    # stretch
    
    chi2_shift_interp = scipy.interpolate.Akima1DInterpolator(alpha, chi2_shift)
    
    alpha_mean = alpha[np.argmin(chi2_shift)]
    
    chi2_shift_stretch = chi2_shift_interp(sigma_alpha * (alpha - alpha_mean) + alpha_mean, extrapolate=False)
    chi2_shift_stretch[np.isnan(chi2_shift_stretch)] = 10**10
    
    return chi2_shift_stretch

# def chi2_shift_stretch(alpha, chi2, delta_alpha=0.0, sigma_alpha=1.0, nan_fill_value=None, method="pchip"):

#     # Select interpolator
#     if method.lower() == "pchip":
#         interpolator = scipy.interpolate.PchipInterpolator(alpha, chi2, extrapolate=(nan_fill_value is None))
#     elif method.lower() == "akima":
#         interpolator = scipy.interpolate.Akima1DInterpolator(alpha, chi2)
#     else:
#         raise ValueError("Method must be 'pchip' or 'akima'")

#     # Apply shift and stretch in alpha-space
#     alpha_shifted = alpha - delta_alpha
#     alpha_min_chi2 = alpha_shifted[np.argmin(interpolator(alpha_shifted))]
#     alpha_transformed = sigma_alpha * (alpha_shifted - alpha_min_chi2) + alpha_min_chi2

#     # Interpolate transformed curve
#     chi2_transformed = interpolator(alpha_transformed)

#     # Handle NaNs
#     if nan_fill_value is not None:
#         chi2_transformed = np.where(np.isnan(chi2_transformed), nan_fill_value, chi2_transformed)

#     return chi2_transformed


In [ ]:
dataset_list = [
    "DESY6", 
    "DESY6_DR1tiles_noDESI", 
    "DESY6_dec_below-23.5"
]

color_dict = {
    "DESY6": "mediumblue",
    "DESY6_DR1tiles_noDESI": "#197D19",
    "DESY6_DR1tiles_DESIonly": "mediumpurple",
    "DESY6_dec_below-23.5": "orange",
    "DESY6_dec_above-23.5": "red"
}

fig, ax = plt.subplots(figsize=(6, 5))

for dataset in dataset_list:
    alpha_vector, chi2_vector_orig = likelihoods[dataset, "y", 1, "[]"].T
    # ax.plot(alpha_vector, chi2_vector_orig - chi2_vector_orig.min(), color=color_dict.get(dataset, "black"), label=fr"rework_dataset_name(dataset)}")

    alpha_orig, sigma_alpha_orig = chi2_sigma_region(alpha_vector, chi2_vector_orig - chi2_vector_orig.min())
    print([dataset, f"{alpha_orig:.4f}", f"{sigma_alpha_orig:.4f}", f"{sigma_alpha_orig:.4f}"])

    alpha_vector_ss = np.copy(alpha_vector)
    chi2_vector_ss = chi2_shift_stretch(alpha_vector, chi2_vector_orig - chi2_vector_orig.min(), sigma_alpha=sigma_alpha_orig / np.sqrt(sigma_alpha_orig**2 + systematic_error**2))
    chi2_vector_ss += chi2_vector_orig.min()
    
    alpha_vector_ss = alpha_vector_ss[np.concatenate(np.argwhere(chi2_vector_ss < 10**3))]
    chi2_vector_ss = chi2_vector_ss[np.concatenate(np.argwhere(chi2_vector_ss < 10**3))]
    
    ax.plot(alpha_vector_ss, chi2_vector_ss - chi2_vector_ss.min(), color=color_dict.get(dataset, "black"), label=fr"{rework_dataset_name(dataset)}")

    alpha_ss, sigma_alpha_ss = chi2_sigma_region(alpha_vector_ss, chi2_vector_ss - chi2_vector_ss.min())
    print([dataset, f"{alpha_ss:.4f}", f"{sigma_alpha_ss:.4f}", f"{np.sqrt(sigma_alpha_orig**2 + systematic_error**2):.4f}"])
    
    alpha_nw, chi2_nw = likelihoods[dataset, "n", 1, "[]"].T
    ax.plot(alpha_nw, chi2_nw - chi2_vector_orig.min(), color=color_dict.get(dataset, "black"), linestyle="--")

    np.savetxt(
        f"plots/likelihood_{dataset}.csv",
        np.column_stack([alpha_vector_ss, chi2_vector_ss]),
        delimiter=",",
        header="alpha,chi2",
        comments="",  # avoids '#' in header
        fmt="%.6e"    # scientific notation, adjust as needed
    )

# Horizontal lines
for n in np.arange(1, 6):
    y = n**2
    ax.axhline(y, color="grey", linestyle="--")
    ax.text(1.21, y, rf"{n}$\sigma$", va='center', ha='left', fontsize=15, color='black')

# Axis limits
ax.set_xlim([0.8, 1.2])

# Labels and ticks
ax.set_xlabel(r"$\alpha$", fontsize=20)
ax.set_ylabel(r"$\chi^2$", fontsize=20)
ax.xaxis.set_major_locator(MaxNLocator(5))
ax.yaxis.set_major_locator(MaxNLocator(5))
ax.tick_params(axis="both", which="major", labelsize=15)

ax.legend(fontsize=13)

plt.savefig("plots/likelihoods_all.pdf", bbox_inches="tight")


In [ ]:
dataset_list = ["DESY6", "DESY6_DR1tiles_noDESI", "DESY6_dec_below-23.5"]
include_wiggles = "y"
weight_type = 1
bins_removed_list = [[1, 2, 3, 4, 5], [0, 2, 3, 4, 5], [0, 1, 3, 4, 5], [0, 1, 2, 4, 5], [0, 1, 2, 3, 5], [0, 1, 2, 3, 4]]

values = [0.65 + 0.1 * i for i in range(len(bins_removed_list))]
z_dict = {str(bins_removed): val for bins_removed, val in zip(bins_removed_list, values)}

color_dict = {
    "DESY6": "mediumblue",
    "DESY6_DR1tiles_noDESI": "#197D19",
    "DESY6_DR1tiles_DESIonly": "mediumpurple",
    "DESY6_dec_below-23.5": "orange",
    "DESY6_dec_above-23.5": "red"
}

fig, ax = plt.subplots()

for i_dataset, dataset in enumerate(dataset_list):
    leg_cont = 0
    x_shift = 0.005 * i_dataset
    for bins_removed in bins_removed_list:
        alpha, sigma_alpha, chi2, dof = fit_results[dataset, include_wiggles, weight_type, str(bins_removed)].T

        if sigma_alpha < 1e3:
            ax.errorbar(
                z_dict[str(bins_removed)] + x_shift,
                alpha,
                sigma_alpha,
                capsize=3,
                fmt=".",
                color=color_dict.get(dataset, "black"),
                label=fr"{rework_dataset_name(dataset)}" if leg_cont == 0 else None  # avoid duplicate legend entries
            )
            leg_cont = 1

ax.set_ylim([0.83, 1.17])

ax.set_xlabel(r"$z$", fontsize=20)
ax.set_ylabel(r"$\alpha$", fontsize=20)
ax.xaxis.set_major_locator(MaxNLocator(5))
ax.yaxis.set_major_locator(MaxNLocator(5))
ax.tick_params(axis="both", which="major", labelsize=15)

ax.axhline(y=1, color="black", linestyle="--")

ax.legend(fontsize=13)

# Add labels for bins
for i, bins_removed in enumerate(bins_removed_list):
    x_pos = z_dict[str(bins_removed)]
    if i != 0:
        ax.text(
            x_pos,
            1.175,  # slightly above the y-limit (adjust if needed)
            fr"bin {i+1}",  # Labels start from bin 2
            ha="center",
            va="bottom",
            fontsize=15
        )

plt.savefig("plots/bao_individual_bins.pdf", bbox_inches="tight")

# bins_removed_list = [[]]

# for i_dataset, dataset in enumerate(dataset_list):
#     x_shift = 0.005 * i_dataset
#     for bins_removed in bins_removed_list:
#         alpha, sigma_alpha, chi2, dof = fit_results[dataset, include_wiggles, weight_type, str(bins_removed)].T

#         if sigma_alpha < 1e3:
#             ax.errorbar(
#                 0.85 + x_shift,
#                 alpha,
#                 sigma_alpha,
#                 capsize=3,
#                 fmt="d",
#                 color=color_dict.get(dataset, "black"),
#             )


In [ ]:
# cosmology_params_planck = CosmologicalParameters("planck", verbose=True)
# cosmo_planck = cosmology_params_planck.get_cosmology()

# DM_planck = cosmo_planck.comoving_angular_distance(0.85) / cosmo_planck.h
# rd_planck = cosmo_planck.rs_drag / cosmo_planck.h
# factor_planck = DM_planck / rd_planck


# cosmology_params_mice = CosmologicalParameters("mice", verbose=True)
# cosmo_mice = cosmology_params_mice.get_cosmology()

# DM_mice = cosmo_mice.comoving_angular_distance(0.85) / cosmo_mice.h
# rd_mice = cosmo_mice.rs_drag / cosmo_mice.h
# factor_mice = DM_mice / rd_mice


# print(factor_mice / factor_planck)
